In [ ]:
!pip install transformers datasets sentencepiece -q

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 23.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 8.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 13.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 8.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 14.3 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.


In [ ]:
!pip install sacremoses

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 897.5/897.5 kB 27.6 MB/s eta 0:00:00


In [ ]:
!pip install datasets

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from google.colab import drive
drive.mount('/content/drive')

file_path = '/content/drive/MyDrive/en-ru_train.csv'
df = pd.read_csv(file_path)

df['en'] = df['translation'].apply(lambda x: eval(x)['en'])
df['ru'] = df['translation'].apply(lambda x: eval(x)['ru'])

train_data, test_data = train_test_split(df, test_size=0.1, random_state=42)
val_data, test_data = train_test_split(test_data, test_size=0.5, random_state=42)

train_data.to_csv('train.csv', index=False)
val_data.to_csv('val.csv', index=False)
test_data.to_csv('test.csv', index=False)


Mounted at /content/drive


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
from datasets import load_dataset

data_files = {
    "train": "train.csv",
    "validation": "val.csv",
    "test": "test.csv"
}
dataset = load_dataset('csv', data_files=data_files)

Generating train split: 0 examples [00:00, ? examples/s]

Generating validation split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

In [ ]:
from transformers import AutoTokenizer

model_name = "Helsinki-NLP/opus-mt-en-ru"
tokenizer = AutoTokenizer.from_pretrained(model_name)

def preprocess_function(examples):
    inputs = examples['en']
    targets = examples['ru']
    model_inputs = tokenizer(inputs, max_length=128, truncation=True, padding="max_length")
    with tokenizer.as_target_tokenizer():
        labels = tokenizer(targets, max_length=128, truncation=True, padding="max_length")

    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

tokenized_datasets = dataset.map(preprocess_function, batched=True)


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/42.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.38k [00:00<?, ?B/s]

source.spm:   0%|          | 0.00/803k [00:00<?, ?B/s]

target.spm:   0%|          | 0.00/1.08M [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/2.60M [00:00<?, ?B/s]

Map:   0%|          | 0/15746 [00:00<?, ? examples/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:3953: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(


Map:   0%|          | 0/875 [00:00<?, ? examples/s]

Map:   0%|          | 0/875 [00:00<?, ? examples/s]

In [ ]:
from transformers import AutoModelForSeq2SeqLM
model = AutoModelForSeq2SeqLM.from_pretrained(model_name)

pytorch_model.bin:   0%|          | 0.00/307M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/293 [00:00<?, ?B/s]

In [ ]:
from transformers import DataCollatorForSeq2Seq
data_collator = DataCollatorForSeq2Seq(tokenizer=tokenizer, model=model)

In [ ]:
from transformers import Seq2SeqTrainingArguments, Seq2SeqTrainer

import numpy as np
training_args = Seq2SeqTrainingArguments(
    output_dir="./results",
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    weight_decay=0.01,
    save_total_limit=3,
    num_train_epochs=3,
    predict_with_generate=True,
    fp16=True
)
trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
    tokenizer=tokenizer,
    data_collator=data_collator,
)
trainer.train()

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
<ipython-input-14-ca89c2f5f8a8>:16: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Seq2SeqTrainer.__init__`. Use `processing_class` instead.
  trainer = Seq2SeqTrainer(
wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


Epoch,Training Loss,Validation Loss
1,0.641000,0.375793
2,0.366300,0.365087
3,0.343100,0.362468


/usr/local/lib/python3.10/dist-packages/transformers/modeling_utils.py:2817: UserWarning: Moving the following attributes in the config to the generation config: {'max_length': 512, 'num_beams': 4, 'bad_words_ids': [[62517]]}. You are seeing this warning because you've set generation parameters in the model config, as opposed to in the generation config.
  warnings.warn(


TrainOutput(global_step=2955, training_loss=0.4115558088531752, metrics={'train_runtime': 550.5727, 'train_samples_per_second': 85.798, 'train_steps_per_second': 5.367, 'total_flos': 1601290962468864.0, 'train_loss': 0.4115558088531752, 'epoch': 3.0})

In [ ]:
results = trainer.evaluate(tokenized_datasets["test"])
print("Test Results:", results)

NameError: name 'trainer' is not defined

In [ ]:
model.save_pretrained("/content/drive/MyDrive/models/en-ru-translation_model")
tokenizer.save_pretrained("/content/drive/MyDrive/models/en-ru-translation_model")

In [ ]:
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer
model_path = "/content/drive/MyDrive/models/en-ru-translation_model"
model = AutoModelForSeq2SeqLM.from_pretrained(model_path)
tokenizer = AutoTokenizer.from_pretrained(model_path)


In [ ]:
def translate(sentence, model, tokenizer, max_length=50):
    inputs = tokenizer(sentence, return_tensors="pt", truncation=True)
    outputs = model.generate(**inputs, max_length=max_length, num_beams=5, early_stopping=True)
    translation = tokenizer.decode(outputs[0], skip_special_tokens=True)
    return translation


In [ ]:
test_sentences = [
    "Hello, how are you?",
    "hi what is your name",
    "I love programming in c.",
    "The weather is nice today.",
    "Can you help me with this task?",
    "This is a wonderful opportunity!"
]
for sentence in test_sentences:
    russian_translation = translate(sentence, model, tokenizer)
    print(f"English: {sentence}")
    print(f"Russian: {russian_translation}\n")


In [ ]:
pip install flask

In [ ]:
pip install gunicorn


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.0/85.0 kB 4.8 MB/s eta 0:00:00


In [ ]:
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity
!pip install streamlit
import streamlit as st
from transformers import MarianMTModel, MarianTokenizer
import torch

# Load MarianMT model and tokenizer
model_path = "/content/drive/MyDrive/models/en-ru-translation_model"  # Path to your fine-tuned MarianMT model

# Load using the local path
model = MarianMTModel.from_pretrained(model_path, local_files_only=True)
tokenizer = MarianTokenizer.from_pretrained(model_path, local_files_only=True)

# Load Sentence-BERT model
sbert_model = SentenceTransformer('paraphrase-MiniLM-L6-v2')

# Translation function
def translate(input_sentence):
    inputs = tokenizer(input_sentence, return_tensors="pt", padding=True, truncation=True)
    with torch.no_grad():
        translated_tokens = model.generate(inputs["input_ids"], max_length=128, num_beams=4, early_stopping=True)
    translated_sentence = tokenizer.decode(translated_tokens[0], skip_special_tokens=True)
    return translated_sentence

# Function to calculate semantic similarity using Sentence-BERT
def calculate_semantic_similarity(original_text, translated_text):
    embeddings = sbert_model.encode([original_text, translated_text])
    similarity = cosine_similarity([embeddings[0]], [embeddings[1]])
    return similarity[0][0]

# Streamlit interface for Sentence-BERT based similarity
st.title("English to russia Translation with MarianMT")
st.write("Enter an English sentence to get its russia translation.")

input_sentence = st.text_input("Enter English sentence:")
if st.button("Translate Sentence"):
    if input_sentence.strip() == "":
        st.warning("Please enter a sentence to translate.")  # Warning for empty input
    else:
        translated_sentence = translate(input_sentence)
        st.write(f"Translated Sentence: {translated_sentence}")

        # Calculate Semantic Similarity between original and translated sentence
        similarity_score = calculate_semantic_similarity(input_sentence, translated_sentence)

        # Display the similarity score
        st.write(f"Semantic Similarity Score: {similarity_score:.4f}")

2024-12-19 05:18:17.088 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-12-19 05:18:17.090 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-12-19 05:18:17.093 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-12-19 05:18:17.095 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-12-19 05:18:17.097 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-12-19 05:18:17.099 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-12-19 05:18:17.100 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-12-19 05:18:17.103 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bar

In [ ]:
!pip install streamlit pyngrok


In [ ]:
!pip uninstall tensorflow -y
!pip install tensorflow

Fatal Python error: init_sys_streams: can't initialize sys standard streams
Python runtime state: core initialized
Traceback (most recent call last):
  File "/usr/lib/python3.10/codecs.py", line 186, in __init__
    def __init__(self, errors='strict'):
KeyboardInterrupt


In [ ]:
!pip install --upgrade pyngrok


In [ ]:
! pip install streamlit -q

In [ ]:
!wget -q -O - ipv4.icanhazip.com

34.16.190.44


In [ ]:
!pip install PyPDF2



In [ ]:
! streamlit run app1.py & npx localtunnel --port 8501




⠙⠹
  You can now view your Streamlit app in your browser.

  Local URL: http://localhost:8501
  Network URL: http://172.28.0.12:8501
  External URL: http://34.16.190.44:8501

⠸⠼⠴⠦your url is: https://wise-places-pull.loca.lt
2024-12-19 05:25:12.533513: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1734585912.575839    8178 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1734585912.586269    8178 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-12-19 05:25:12.625742: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following

# New Section

In [ ]:
sbert_model = SentenceTransformer('paraphrase-mpnet-base-v2')


In [ ]:
translated_tokens = model.generate(inputs["input_ids"], max_length=128, num_beams=4, early_stopping=True)


In [ ]:
# Load a larger Sentence-BERT model
sbert_model = SentenceTransformer('paraphrase-mpnet-base-v2')

# Translation function with a larger beam search width
def translate(input_sentence):
    inputs = tokenizer(input_sentence, return_tensors="pt", padding=True, truncation=True)
    with torch.no_grad():
        translated_tokens = model.generate(inputs["input_ids"], max_length=128, num_beams=8, early_stopping=True)
    translated_sentence = tokenizer.decode(translated_tokens[0], skip_special_tokens=True)
    return translated_sentence
